In [1]:
import torch
import torch.nn as nn
from transformers import BertTokenizer, BertModel
import pandas as pd
from tqdm import tqdm

In [2]:
class BERTClassifier(nn.Module):
	def __init__(self, bert_model_name, num_classes):
		super(BERTClassifier, self).__init__()
		self.bert = BertModel.from_pretrained(bert_model_name)
		self.dropout = nn.Dropout(0.1)
		self.fc = nn.Linear(self.bert.config.hidden_size, num_classes)
		
	def forward(self, input_ids, attention_mask):
		outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
		pooled_output = outputs.pooler_output
		x = self.dropout(pooled_output)
		logits = self.fc(x)
		return logits

In [ ]:
polyhope_multi_labels = {'Not Hope':0, 'Generalized Hope':1, 'Realistic Hope':2, 'Unrealistic Hope':3}
polyhope_inv_multi_labels = {v: k for k, v in polyhope_multi_labels.items()}
polyhope_binary_labels = {'Hope':1, 'Not Hope':0}
polyhope_inv_binary_labels = {v: k for k, v in polyhope_binary_labels.items()}

In [3]:
#'dccuchile/bert-base-spanish-wwm-cased'
model_name = 'bert-base-uncased' 
num_classes = 4
device = torch.device('cpu')

tokenizer = BertTokenizer.from_pretrained(model_name)

model = BERTClassifier(model_name, num_classes)
model.load_state_dict(torch.load('../models/poly_multi_english_uncased.pth', map_location=device))

Some weights of BertModel were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-cased and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
def predict_sentiment(text, label_map, model, tokenizer, device, max_length=128):
    model.eval()
    encoding = tokenizer(text, return_tensors='pt', max_length=max_length, padding='max_length', truncation=True)
    input_ids = encoding['input_ids']
    attention_mask = encoding['attention_mask']

    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        _, preds = torch.max(outputs, dim=1)
        return label_map[preds.item()]

In [5]:
# test = 'this is a good example'
# predict_sentiment(test, model, tokenizer, device)

In [6]:
def get_embeddings(text, model, tokenizer, device, max_length=128):
    model.eval()
    encoding = tokenizer(text, return_tensors='pt', max_length=max_length, padding='max_length', truncation=True)
    input_ids = encoding['input_ids']
    attention_mask = encoding['attention_mask']

    with torch.no_grad():
        outputs = model.bert(input_ids=input_ids, attention_mask=attention_mask)
        return outputs.pooler_output

In [9]:
df = pd.read_csv('../data/val_polyhope_english.csv')
actual_predictions = df['category'].tolist()
texts = df['text'].tolist()

for i, text in enumerate(tqdm(texts)):
    embeddings.append(get_embeddings(text, model, tokenizer, device)[0].numpy())
    pred.append(predict_sentiment(text, polyhope_inv_multi_labels, model, tokenizer, device))



100%|██████████████████████████████████████████████████████████████| 200/200 [00:42<00:00,  4.67it/s]
